In [231]:
import pandas as pd 
import utils.preprocessing

df0 = pd.read_csv('data/raw/offenseval-training-v1.tsv', sep='\t')
utils.preprocessing.clean(df0)
df0.head()

,id,tweet,subtask_a,subtask_b,subtask_c,clean_tweets
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,ask native americans
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,home drunk maga trump
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,amazon investigating chinese employees selling...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,someone vetaken piece shit volcano
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,obama wanted liberals amp illegals move red s...


In [232]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy import sparse

import numpy as np

In [255]:
vec = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000)

In [233]:
from nltk.stem.porter import *
from nltk.tokenize import TweetTokenizer

stemmer = PorterStemmer()
vec = CountVectorizer(ngram_range=(1,2))

In [256]:
df = df0.copy()
train_size = int(len(df)*0.8)
#train_size = len(df_crowd)+int(len(df0)*0.8)

y = df["subtask_a"].map({"OFF":0, "NOT":1})
y_train = y[:train_size]
y_test = y[train_size:]

X_train = vec.fit_transform(df["clean_tweets"][:train_size])
X_test = vec.transform(df["clean_tweets"][train_size:])

X_train, y_train = check_X_y(X_train, y_train, accept_sparse=True)

### Naive Bayes

In [257]:
p = X_train[y_train==1].sum(0)+1
q = X_train[y_train==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(X_train[y_train==1].shape[1])/(X_train[y_train==0].shape[0])

In [258]:
preds = X_test @ r.T + b
y_pred = [1 if x>0 else 0 for x in np.array(preds.T)[0]]
accuracy_score(y_pred, y_test)

0.7356495468277946

In [259]:
clf = LogisticRegression(C=4, dual=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7651057401812689

In [260]:
clf = LogisticRegression(C=4, dual=True)
clf.fit(X_train @ r.T, y_train)
y_pred = clf.predict(X_test @ r.T)
accuracy_score(y_pred, y_test)

0.7462235649546828

In [242]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7643504531722054

In [28]:
df_crowd = pd.read_csv('data/additional/labeled_data.csv', usecols=[5,6])
df_crowd.columns = ["subtask_a","tweet"]
df_crowd = df_crowd.reindex(['tweet', 'subtask_a'], axis=1)
df_crowd["new_id"] = pd.Series(["crowd_"+str(x) for x in df_crowd.index])
df_crowd = df_crowd.set_index("new_id")

df_crowd = df_crowd[df_crowd["subtask_a"]==0]
mapping = {2:"NOT", 1:"OFF", 0:"OFF"}
df_crowd["subtask_a"] = df_crowd["subtask_a"].map(mapping)
#df_crowd["subtask_a"].map({2:"neither", 1:"offensive", 0:"hate speech"}).value_counts()

utils.preprocessing.clean(df_crowd)
df = pd.concat([df_crowd, df0[df_crowd.columns]])
df.head()

,tweet,subtask_a,clean_tweets
crowd_1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,OFF,boy dats cold tyga dwn bad cuffin dat hoe place
crowd_2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,OFF,dawg ever fuck bitch start cry confused shit
crowd_3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,OFF,look tranny
crowd_4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,OFF,shit hear true faker bitch told
crowd_5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",OFF,shit blows claim faithful somebody fucking hoes
